In [1]:
%cd ~/cdv/

import os

os.environ['LD_LIBRARY_PATH'] = '/home/nmiklaucic/nccl_2.21.5-1+cuda11.0_x86_64/lib:' + os.environ['LD_LIBRARY_PATH']

/home/nmiklaucic/cdv


/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import jax 
import jax.numpy as jnp
import jax.random as jr

In [3]:
from jax.experimental import mesh_utils
from jax.sharding import Mesh, PartitionSpec as P, NamedSharding

d = 3
mesh = Mesh(mesh_utils.create_device_mesh((d,), devices=jax.devices()[:d]), 'batch')
sharding = NamedSharding(mesh, P('batch', None))

In [4]:
tree = {
    'a': jr.normal(jr.key(123), (3, 64)),
    'b': jr.normal(jr.key(123), (3, 32, 32)),
    'c': jr.normal(jr.key(123), (3, 1))
}

tree = jax.device_put(tree, sharding)

jax.debug.visualize_array_sharding(tree['a'][0])

E0816 16:33:08.249164 1082658 pjrt_stream_executor_client.cc:2804] Execution of replica 0 failed: INTERNAL: external/xla/xla/service/gpu/nccl_api.cc:395: NCCL operation ncclCommInitRankConfig(&comm_handle, nranks, AsNcclUniqueId(clique_id), ranks[i].rank, &comm_config) failed: unhandled system error. Last NCCL warning(error) log entry (may be unrelated) 'Unable to load NCCL library. Multi-GPU collectives will not work.'.
E0816 16:33:08.249187 1082656 pjrt_stream_executor_client.cc:2804] Execution of replica 0 failed: INTERNAL: external/xla/xla/service/gpu/nccl_api.cc:395: NCCL operation ncclCommInitRankConfig(&comm_handle, nranks, AsNcclUniqueId(clique_id), ranks[i].rank, &comm_config) failed: unhandled system error. Last NCCL warning(error) log entry (may be unrelated) 'Unable to load NCCL library. Multi-GPU collectives will not work.'.
E0816 16:33:08.249202 1082654 pjrt_stream_executor_client.cc:2804] Execution of replica 0 failed: INTERNAL: external/xla/xla/service/gpu/nccl_api.cc:3

XlaRuntimeError: INTERNAL: external/xla/xla/service/gpu/nccl_api.cc:395: NCCL operation ncclCommInitRankConfig(&comm_handle, nranks, AsNcclUniqueId(clique_id), ranks[i].rank, &comm_config) failed: unhandled system error. Last NCCL warning(error) log entry (may be unrelated) 'Unable to load NCCL library. Multi-GPU collectives will not work.'.: while running replica 0 and partition 0 of a replicated computation (other replicas may have failed as well).

: 